In [ ]:
import requests, re
# import json
import sqlite3
import nltk
# import multiprocessing
# import threading
# import time
import pandas as pd

In [ ]:
# ALL FUNCTION HERE
end_points = 'https://voiceviet.itrithuc.vn/api/v1/' # API ENDPOINT
api_key = '812f2448624c42899fbf794f54f591f9' # API KEY

# GET SESSION KEY
def get_session():
    login_url = end_points + 'user/login/speech@usth.edu.vn/c950f8a82221d74a357af014434de14cb9dc11850b8ecc2a356dd4c945df2518'
    session = requests.session()
    headers = {'Authorization-Key': api_key}
    r = session.get(login_url, headers=headers)
    data = r.json()
    resp = data['resp']
    return session, resp['token']

# POST TEXT by API
def post_text(text,cId):
    session, token = get_session()
    text = {"text": text}
    payload = json.dumps(text,ensure_ascii=False).encode('utf8')
    token = 'Bearer: ' + token
    url = end_points + 'text/write/'+str(cId)+'/1'
    headers = {'Authorization-Key': api_key, 'Authorization': token, 'Content-type': 'application/json'}
    r = session.post(url, data=payload, headers=headers)
    return r.text

# GET RANDOM TEXT by API
def get_text(tid):
    url = end_points + 'text/' + str(tid)
    headers = {'Authorization-Key': api_key}
    r = requests.get(url, headers=headers)
    return r

def get_random():
    url = end_points + 'text/random'
    headers = {'Authorization-Key': api_key}
    r = requests.get(url, headers=headers)
    return r

def get_domain(pS,pN):
    url = end_points + 'domain/list/' + str(pS) +'/'+ str(pN)
    headers = {'Authorization-Key': api_key}
    r = requests.get(url, headers=headers)
    return r
    
def post_category(name):
    session, token = get_session()
    url = end_points + 'category/'
    token = 'Bearer: ' + token
    payload = {'name': name, 'domainId': 8}
    headers = {'Authorization-Key': api_key, 'Authorization': token}
    r = requests.post(url, json=payload, headers=headers)
    return r.text

def get_category(pS,pN):
    url = end_points + 'category/list/' + str(pS) +'/'+ str(pN)
    headers = {'Authorization-Key': api_key}
    r = requests.get(url, headers=headers)
    return r

def get_cat_text(cId, pS, pN):
    session, token = get_session()
    url = end_points + 'category/'+str(cId)+'/text/' + str(pS) +'/'+ str(pN)
    headers = {'Authorization-Key': api_key, 'Authorization': token}
    r = session.get(url, headers=headers)
    return url    

# FUNCTION TO CONNECT TO DATABASE
# def create_connection():
#     conn = sqlite3.connect("new_mt.db")
#     curr = self.conn.cursor()

# # FUNCTION TO CREATE TABLE
# def create_table():
#     curr.execute("""DROP TABLE IF EXISTS summary""")
#     curr.execute("""create table summary(id integer primary key, summary text unique, tag text)""")

# FUNCTION TO CLEAN TEXT
def clean_text(text):
#         print('HERE: ',len(text))
    text = re.sub('–','',text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub('\W+',' ', text)
    text = re.sub(' +',' ',text)
    text = text.lstrip()
    text = text.rstrip()
    text = text.lower()
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    return text

# # FUNCTION TO STORE DATA TO DATABASE
# def store_db(self, item):
#     split_text = item['summary'][0].split(". ") 
#     for i in range(len(split_text)):
# #               print("HERE: ", split_text[i], item['tag'])
#         self.curr.execute("""insert or ignore into summary (summary, tag) values (?,?)""",(split_text[i], item['tag']))
#         self.conn.commit()

In [ ]:
tag = "Random"
conn = sqlite3.connect("random.db")
curr = conn.cursor()
curr.execute("SELECT * FROM summary WHERE tag = ?",(tag,))
for row in curr:
    post_text(row[1],150)
    print(row[0])
# for i in range(len(datastore)):
#     if len(datastore[i]['summary']) > 1:
#         datastore[i]['summary'] = [' '.join(datastore[i]['summary'])]
#         datastore[i]['summary'][0] = datastore[i]['summary'][0].replace('\xa0','')
# # print(datastore[0]['summary'][0])
#     post_text(datastore[i]['summary'][0],150)

In [ ]:
r = get_text(148200)
json_data = json.loads(r.text)
# data = json_data['resp']
# data = data['text']
# data = data.decode('utf-8')
json_data

In [ ]:
get_category(150,1).text

In [ ]:
get_category(150,1).text

In [ ]:
post_text("UBND TP.HCM cho biết, vừa qua TP đã yêu cầu các sở-ngành, UBND quận-huyện tuyên truyền, vận động, hướng dẫn người dân, hộ gia đình, chủ nguồn thải thực hiện phân loại chất thải rắn sinh hoạt tại nguồn từ nay cho đến hết quý 2/2019.",147)

In [ ]:
get_text(148500).text

In [ ]:
import ipdb
def Get_raw_text(threadName):
    conn = sqlite3.connect("random.db")
    curr = conn.cursor()
#     print('Start')
    i=0
    while i<50000:
        try:
            r = get_random()
            json_data = json.loads(r.text)
            text = json_data['resp']['text']
            curr.execute("""insert or ignore into raw (text) values (?)""",(text,))
            conn.commit()
            print(threadName + ": " + str(i))
            i += 1
        except:
            continue
def Get_text(df,words):
#     ipdb.set_trace()
    print("Hello")
    conn = sqlite3.connect("random.db")
    curr = conn.cursor()
#     curr.execute("SELECT * from raw")
#     result = curr.fetchall()
#     df = pd.DataFrame(columns=['raw_text'])
#     for i in result:
#         row = [i[1]]
#         df.loc[len(df)] = row
    for i in range(len(df)):
        text = df.text[i]
        text = clean_text(text)       
        text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w in words)
        text = '<s> '+text+' </s>'
        curr.execute("""insert or ignore into summary (text) values (?)""",(text,))
        conn.commit()
        print(str(i) + text)

In [ ]:
import traceback
with open('Viet74K.txt') as f:
    lines = f.read().splitlines()
lines
one_lines = ''
for i in range(len(lines)):
    one_lines += ' ' + lines[i]
words = set(one_lines.split())

In [ ]:
# words
# conn = sqlite3.connect("random.db")
# curr = conn.cursor()
# curr.execute("""DROP TABLE IF EXISTS raw""")
# curr.execute("""create table raw(id integer primary key, text text unique)""")
# curr.close()
# conn.close()

class myThread (threading.Thread):
    def __init__(self, name):
        threading.Thread.__init__(self)
        self.name = name
    def run(self):
        print ("Starting " + self.name)
        Get_raw_text(self.name)
        print ("Exiting " + self.name)

threads = []
# Create new threads
for m in range(1,15):
    thread = myThread("Thread-"+str(m))
    threads.append(thread)

# Start new Threads
for t in threads:
    t.start()
for t in threads:
    t.join()
# thread1.start()
# thread2.start()
# thread1.join()
# thread2.join()
print ("Exiting Main Thread")
# _thread.start_new_thread

In [ ]:
conn = sqlite3.connect('./random.db')
curr = conn.cursor()
curr.execute("SELECT * from summary WHERE id > 57577")
result = curr.fetchall()
df = pd.DataFrame(columns=['text'])
for i in result:
    row = [i[1]]
    df.loc[len(df)] = row

In [ ]:
# 57577
Get_text(df,words)

In [ ]:
with open('plus++_text.txt', 'w') as f:
    for i in range(len(df)):
        f.write(df.text[i]+'\n')

In [ ]:
df